In [1]:
# Importing Required libraries

from lxml import html
from json import dump, loads
from requests import get
import json
from re import sub
from dateutil import parser as dateparser_to_html
# from time import sleep
import urllib3
import glob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import re
import requests
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.remove('no')
stop_words.remove('not')

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nose\config.py:178: DeprecationWarning: invalid escape sequence \.
  """


## Scraping Amazon Asin numbers       

In [ ]:
# urls = [
#     'https://www.amazon.in/gp/browse.html?node=1389396031&ref_=nav_em_T1_0_4_NaN_1_sbc_tvelec_television'
# ]

# for url in urls:
#     content = requests.get(url).content
#     decoded_content = content.decode()
    
#     asins = set(re.findall(r'/[^/]+/dp/([^"?]+)', decoded_content))
#     asin_numbers = []
#     for i in asins:
#         if len(i) == 10:
#             asin_numbers.append(i)

# print(asin_numbers)  

## Amazon Television Asin numbers

In [23]:
# asin_numbers = ['B07GTD1LHY', 'B07T89Z35Z', 'B07QB77C92', 'B07NSLTQH6', 'B07DTSCMMB', 'B06XGWRKYT', 'B07TMFQMJC', 'B079ZSFJRG',
#                 'B07GT2P8ST', 'B07MGQS2M8', 'B07TNVR737', 'B07H9LS8DM', 'B07Q6YCJB4', 'B07GBDVNVJ', 'B07RFWXM96', 'B07R72W9K8',
#                 'B07XMD275Q', 'B07W5QZQ45', 'B07X413J1G', 'B07QD64TL4', 'B07XMD275S', 'B07RJQYL97', 'B07MNF89L2', 'B07TJ873DB',
#                 'B07X841RGL', 'B07W5GNXKH', 'B07HB7SSPZ', 'B07XK9JSGY', 'B081Z1RQLV', 'B07F1SKCZ2', 'B07H7WPV7D', 'B072QD8QKL']

## Scraping Amazon product reviews

In [22]:
# def get_header(asin):
#     try:
#         ratings_dict = {}
#         amazon_url = 'https://www.amazon.in/product-reviews/' + asin + '/ref=cm_cr_arp_d_paging_btm_next_1?pageNumber=1'
#         urllib3.disable_warnings()
#         headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
#         response = get(amazon_url, headers=headers, verify=False, timeout=30)
#         # Removing the null bytes from the response.
#         cleaned_response = response.text.replace('\x00', '')
#         parser_to_html = html.fromstring(cleaned_response)

#         number_reviews = ''.join(parser_to_html.xpath('.//span[@data-hook="total-review-count"]//text()')).replace(',', '')
#         product_price = ''.join(parser_to_html.xpath('.//span[contains(@class,"a-color-price arp-price")]//text()')).strip()
#         product_name = ''.join(parser_to_html.xpath('.//a[@data-hook="product-link"]//text()')).strip()
#         total_ratings = parser_to_html.xpath('//table[@id="histogramTable"]//tr')
#         for ratings in total_ratings:
#             extracted_rating = ratings.xpath('./td//a//text()')
#             if extracted_rating:
#                 rating_key = extracted_rating[0]
#                 raw_raing_value = extracted_rating[1]
#                 rating_value = raw_raing_value
#                 if rating_key:
#                     ratings_dict.update({rating_key: rating_value})

#         number_page_reviews = int(int(number_reviews) / 10)

#         if number_page_reviews % 2 == 0:
#             number_page_reviews += 1
#         else:
#             number_page_reviews += 2

#         return product_price, product_name, number_reviews, ratings_dict, number_page_reviews
#     except Exception as e:
#         return {"url": amazon_url, "error": e}


# def get_all_reviews(asin):
#     review_total_pages = []
#     review_total_pages2 = []

#     product_price, product_name = get_header(asin)
#     number_reviews, ratings_dict = get_header(asin)
#     number_page_reviews = get_header(asin)
#     for page_number in range(1, 200):
#         print(page_number)
#         try:
#             amazon_url = 'https://www.amazon.in/product-reviews/' + asin + '/ref=cm_cr_arp_d_paging_btm_next_' \
#                          + str(page_number) + '?pageNumber=' + str(page_number)
#             urllib3.disable_warnings()
#             headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
#             response = get(amazon_url, headers=headers, verify=False, timeout=30)

#             # Removing the null bytes from the response.
#             cleaned_response = response.text.replace('\x00', '')
#             parser_to_html = html.fromstring(cleaned_response)
#         except Exception as e:
#             return {"url": amazon_url, "error": e}
#         reviews = parser_to_html.xpath('//div[contains(@id,"reviews-summary")]')

#         if not reviews:
#             reviews = parser_to_html.xpath('//div[@data-hook="review"]')

#         for review in reviews:
#             raw_review_author = review.xpath('.//span[contains(@class,"profile-name")]//text()')
#             raw_review_rating = review.xpath('.//i[@data-hook="review-star-rating"]//text()')
#             raw_review_header = review.xpath('.//a[@data-hook="review-title"]//text()')
#             raw_review_posted_date = review.xpath('.//span[@data-hook="review-date"]//text()')
#             raw_review_text1 = review.xpath('.//span[@data-hook="review-body"]//text()')
#             raw_review_text2 = review.xpath('.//div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview')
#             raw_review_text3 = review.xpath('.//div[contains(@id,"dpReviews")]/div/text()')

#             # Cleaning data
#             author = ' '.join(' '.join(raw_review_author).split())
#             review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')
#             review_header = ' '.join(' '.join(raw_review_header).split())
#             try:
#                 review_posted_date = dateparser_to_html.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
#             except:
#                 review_posted_date = None
#             review_text = ' '.join(' '.join(raw_review_text1).split())

#             # Grabbing hidden comments if present
#             if raw_review_text2:
#                 json_loaded_review_data = loads(raw_review_text2[0])
#                 json_loaded_review_data_text = json_loaded_review_data['rest']
#                 cleaned_json_loaded_review_data_text = re.sub('<.*?>', '', json_loaded_review_data_text)
#                 full_review_text = review_text + cleaned_json_loaded_review_data_text
#             else:
#                 full_review_text = review_text
#             if not raw_review_text1:
#                 full_review_text = ' '.join(' '.join(raw_review_text3).split())

#             review_total_pages.append(full_review_text)
#             review_total_pages2.append(review_header)

#     data = {
#         'reviews_text': review_total_pages,
#         'reviews_header': review_total_pages2,
#     }
#     return data


# def core():
#     asin_list = asin_numbers
#     for asin in asin_list:
#         print(f"IN PROCESS FOR: {asin}")
#         temp = get_all_reviews(asin)
#         df = pd.DataFrame.from_dict(temp)

#         df['Asin'] = asin
#         df.to_csv(asin + '.csv', index=False)
        
# if __name__ == '__main__':
#     core()

IN PROCESS FOR: B07GTD1LHY
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
IN PROCESS FOR: B07T89Z35Z
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
IN PROCESS FOR: B07Q6YCJB4
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
1

186
187
188
189
190
191
192
193
194
195
196
197
198
199
IN PROCESS FOR: B07TJ873DB
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
IN PROCESS FOR: B07X841RGL
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
7

In [17]:
# # get data file names
# filenames = glob.glob('./data' +"/*.csv")

# dfs = []
# for filename in filenames:
#     dfs.append(pd.read_csv(filename))

# # Concatenate all data into one DataFrame
# df = pd.concat(dfs, ignore_index=True)
# df.to_csv('AmazonData.csv', index=False)

In [18]:
df = pd.read_csv('AmazonData.csv')
df.head()

,reviews_text,reviews_header,Asin
0,"Today, it got delivered. Talked to Amazon Cust...",Cheap Product with Cheap customer service by A...,B06XGWRKYT
1,Got it for 20500. First order had to be replac...,Fits the value.,B06XGWRKYT
2,Great value and quality compared to other TV b...,Great Value !!!,B06XGWRKYT
3,This TV maybe LED as in 'technically LED'. The...,Might do for an urgent need. Spend more for a ...,B06XGWRKYT
4,I would request everyone please don't buy this...,Worst product...don't purchase it,B06XGWRKYT


# Topic Modelling

In [4]:
# Convert to list
data = df.reviews_text.values.tolist()
data

["Today, it got delivered. Talked to Amazon Customer Service - Due to its cheap build-up quality, I wanted to return this TV (even at this price range its pathetic). Amazon denied to return it. This is the first time(in last 3 years), it ever happened with me on Amazon. I feel cheated and I will make sure Amazon will also feel the Pain. They aren't the only one in the Market.",
 'Got it for 20500. First order had to be replaced because of broken screen. Volume is good. Tv is good enough for this price range. Simplistic remote. Plays most formats. Usb are at side in back. Hdmi at the back. Bought it to play my PS3. So no complaints there. Got wall mount but prefer the stand. When I played some movies from blue ray format via USB, the clarity is not that great. But TV HD is good enough as any other TV. Full value for money. Bit heavy at 10 kg. My needs are met so very happy.',
 'Great value and quality compared to other TV brands that are in the market. So far very satisfied with deliver

## Tokenize words and clean text

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['today', 'it', 'got', 'delivered', 'talked', 'to', 'amazon', 'customer', 'service', 'due', 'to', 'its', 'cheap', 'build', 'up', 'quality', 'wanted', 'to', 'return', 'this', 'tv', 'even', 'at', 'this', 'price', 'range', 'its', 'pathetic', 'amazon', 'denied', 'to', 'return', 'it', 'this', 'is', 'the', 'first', 'time', 'in', 'last', 'years', 'it', 'ever', 'happened', 'with', 'me', 'on', 'amazon', 'feel', 'cheated', 'and', 'will', 'make', 'sure', 'amazon', 'will', 'also', 'feel', 'the', 'pain', 'they', 'aren', 'the', 'only', 'one', 'in', 'the', 'market']]


## Creating Bigram & Trigram Models

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['today', 'it', 'got', 'delivered', 'talked', 'to', 'amazon', 'customer', 'service', 'due', 'to', 'its', 'cheap', 'build', 'up', 'quality', 'wanted', 'to', 'return', 'this', 'tv', 'even', 'at', 'this', 'price', 'range', 'its', 'pathetic', 'amazon', 'denied', 'to', 'return', 'it', 'this', 'is', 'the', 'first', 'time', 'in', 'last', 'years', 'it', 'ever', 'happened', 'with', 'me', 'on', 'amazon', 'feel', 'cheated', 'and', 'will', 'make', 'sure', 'amazon', 'will', 'also', 'feel', 'the', 'pain', 'they', 'aren', 'the', 'only', 'one', 'in', 'the', 'market']


## Remove Stopwords, Make Bigrams and Lemmatize

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['today', 'get', 'deliver', 'talk', 'customer', 'service', 'due', 'cheap', 'build', 'quality', 'want', 'return', 'tv', 'even', 'price', 'range', 'pathetic', 'deny', 'return', 'first', 'time', 'last', 'year', 'ever', 'happen', 'feel', 'cheat', 'make', 'sure', 'also', 'feel', 'pain', 'market']]


## Create the Dictionary and Corpus needed for Topic Modeling

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]]


In [10]:
id2word[0]

'also'

In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('also', 1),
  ('build', 1),
  ('cheap', 1),
  ('cheat', 1),
  ('customer', 1),
  ('deliver', 1),
  ('deny', 1),
  ('due', 1),
  ('even', 1),
  ('ever', 1),
  ('feel', 2),
  ('first', 1),
  ('get', 1),
  ('happen', 1),
  ('last', 1),
  ('make', 1),
  ('market', 1),
  ('pain', 1),
  ('pathetic', 1),
  ('price', 1),
  ('quality', 1),
  ('range', 1),
  ('return', 2),
  ('service', 1),
  ('sure', 1),
  ('talk', 1),
  ('time', 1),
  ('today', 1),
  ('tv', 1),
  ('want', 1),
  ('year', 1)]]

## Building the topic model

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## Topics in LDA model

In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.536*"good" + 0.252*"product" + 0.071*"also" + 0.048*"overall" + '
  '0.033*"voice" + 0.030*"amazing" + 0.007*"quite" + 0.005*"extremely" + '
  '0.003*"believe" + 0.002*"piece"'),
 (1,
  '0.345*"money" + 0.264*"value" + 0.176*"excellent" + 0.159*"awesome" + '
  '0.011*"bottom" + 0.003*"sufficient" + 0.000*"app" + 0.000*"prime" + '
  '0.000*"android" + 0.000*"vu"'),
 (2,
  '0.607*"price" + 0.142*"range" + 0.098*"happy" + 0.059*"point" + '
  '0.029*"decent" + 0.010*"basic" + 0.000*"app" + 0.000*"prime" + '
  '0.000*"android" + 0.000*"vu"'),
 (3,
  '0.493*"nice" + 0.232*"worth" + 0.038*"replacement" + 0.033*"right" + '
  '0.028*"cheap" + 0.021*"consider" + 0.021*"pic" + 0.020*"simple" + '
  '0.018*"struggle" + 0.018*"true"'),
 (4,
  '0.199*"time" + 0.139*"screen" + 0.096*"do" + 0.071*"performance" + '
  '0.067*"poor" + 0.062*"install" + 0.053*"return" + 0.050*"ok" + 0.043*"help" '
  '+ 0.033*"ever"'),
 (5,
  '0.117*"issue" + 0.062*"experience" + 0.047*"even" + 0.036*"review" + '


## Compute Model Perplexity and Coherence Score

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.25234173963484

Coherence Score:  0.5113763058349734


## Visualize the topics-keywords

In [15]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.287061 -0.338499       1        1  14.298405
16    -0.266309  0.345085       2        1  11.587327
5     -0.105474  0.008825       3        1   8.849881
8     -0.134598  0.012795       4        1   8.272850
11    -0.057903  0.003283       5        1   5.414993
18     0.010920 -0.000873       6        1   4.846109
6      0.022579 -0.001472       7        1   4.691910
19    -0.008125  0.000100       8        1   4.602105
7      0.046237 -0.002175       9        1   3.876327
14     0.043734 -0.002187      10        1   3.786795
4      0.059660 -0.002550      11        1   3.710135
2      0.063899 -0.002496      12        1   3.493498
1      0.008319 -0.000479      13        1   3.490963
10     0.045671 -0.002120      14        1   3.428476
13     0.095495 -0.003545      15        1   3.340387
12     0.048757 -0.002063      16        1   3.324168
9      0.037978 -0.001671      17        1   3.278630
17     0.070551 -0.002784      18        1   3.182187
3      0.119487 -0.003894      19        1   2.502038
15     0.186183 -0.003282      20        1   0.022819, topic_info=     Category          Freq              Term         Total  loglift  logprob
28    Default  15500.000000                tv  15500.000000  30.0000  30.0000
40    Default  18805.000000              good  18805.000000  29.0000  29.0000
20    Default  11600.000000           quality  11600.000000  28.0000  28.0000
156   Default   8226.000000             sound   8226.000000  27.0000  27.0000
19    Default   5208.000000             price   5208.000000  26.0000  26.0000
185   Default   8843.000000           product   8843.000000  25.0000  25.0000
192   Default   4963.000000              work   4963.000000  24.0000  24.0000
141   Default   6879.000000           picture   6879.000000  23.0000  23.0000
76    Default   3846.000000      installation   3846.000000  22.0000  22.0000
385   Default   3030.000000              nice   3030.000000  21.0000  21.0000
34    Default   3457.000000               buy   3457.000000  20.0000  20.0000
48    Default   2953.000000             money   2953.000000  19.0000  19.0000
41    Default   2584.000000             great   2584.000000  18.0000  18.0000
353   Default   2427.000000             video   2427.000000  17.0000  17.0000
12    Default   2587.000000               get   2587.000000  16.0000  16.0000
236   Default   2289.000000           feature   2289.000000  15.0000  15.0000
62    Default   2260.000000             value   2260.000000  14.0000  14.0000
55    Default   2116.000000            remote   2116.000000  13.0000  13.0000
179   Default   1859.000000                go   1859.000000  12.0000  12.0000
123   Default   2548.000000             issue   2548.000000  11.0000  11.0000
26    Default   1815.000000              time   1815.000000  10.0000  10.0000
171   Default   1773.000000            update   1773.000000   9.0000   9.0000
23    Default   1814.000000           service   1814.000000   8.0000   8.0000
392   Default   1427.000000             worth   1427.000000   7.0000   7.0000
279   Default   1792.000000           support   1792.000000   6.0000   6.0000
397   Default   1506.000000         excellent   1506.000000   5.0000   5.0000
86    Default   1460.000000               use   1460.000000   4.0000   4.0000
172   Default   1450.000000              well   1450.000000   3.0000   3.0000
0     Default   2500.000000              also   2500.000000   2.0000   2.0000
195   Default   1551.000000               day   1551.000000   1.0000   1.0000
...       ...           ...               ...           ...      ...      ...
2623  Topic20      0.008021  reliance_digital      1.189755   3.3859  -8.8512
2602  Topic20      0.008021               log      1.189796   3.3858  -8.8512
2603  Topic20      0.008021          drawback      1.189814   3.3858  -8.8512
2330  Topic20      0.008021         ins